<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# [Ateliers: Technologies des grosses data](https://github.com/wikistat/Ateliers-Big-Data)

# Recommandation de Films par Filtrage Collaboratif: avec <a href="https://cran.r-project.org/"><img src="https://cran.r-project.org/Rlogo.svg" style="max-width: 40px; display: inline" alt="R"/></a> et [softImpute](https://cran.r-project.org/web/packages/softImpute/index.html)

## 1. Introduction

Ce calepin traite d'un problème classique de recommandation par filtrage collaboratif. Le problème général est décrit en [introduction](https://github.com/wikistat/Ateliers-Big-Data/tree/master/3-MovieLens) et dans une [vignette](http://wikistat.fr/pdf/st-m-datSc3-colFil.pdf) de [Wikistat](http://wikistat.fr/). Il est appliqué aux données publiques du site [GroupLens](http://grouplens.org/datasets/movielens/). L'objectif est de tester les méthodes et la procédure d'optimisation sur le plus petit jeu de données composé de 100k notes  de 943 clients sur 1682 films où chaque client a au moins noté 20 films. Les jeux de données plus gros (1M, 10M, 20M notes) peuvent être utilisés pour "passer à l'échelle volume". 


L'objectif est d'utiliser ces seules données pour proposer des recommandations.  Les données initiales sont sous la forme d'une matrice **très creuse** (*sparse*) contenant des notes ou évaluations. **Attention**, les "0" de la matrice ne sont pas des notes mais des *données manquantes*, le film n'a pas encore été vu ou évalué. 

Ce calepin utilise un algorithme satisfaisant à l'objectif de *complétion de grande matrice creuse* implémenté dans la librairie [softImpute de R](https://cran.r-project.org/web/packages/softImpute/index.html). Un [autre calepin](https://github.com/wikistat/Ateliers-Big-Data/blob/master/3-MovieLens/Atelier-MovieLens-pyspark.ipynb) utilise la version de [NMF](http://wikistat.fr/pdf/st-m-explo-nmf.pdf) de [MLlib de Spark](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS) qui autorise permet également la complétion.

En revanche,la version  de NMF incluse dans la librairie [Scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html) traite également des [matrices creuses](http://docs.scipy.org/doc/scipy/reference/sparse.html) mais le critère (moindres carrés) optimisé considère les "0" comme des notes nulles, pas comme des données manquantes. *Elle n'est pas adaptée au problème de complétion*, contrairement à celle de MLliB. Il faudrait sans doute utiliser la librairie [nonnegfac](https://github.com/kimjingu/nonnegfac-python) en Python  de [Kim et al. (2014)](http://link.springer.com/content/pdf/10.1007%2Fs10898-013-0035-4.pdf); **à tester**!

Dans la première partie, le plus petit fichier est partagé en trois échantillons: apprentissage, validation et test; l'optimisation du rang de la factorisation (nombre de facteurs latents) est réalisée par minimisation de l'erreur estimée sur l'échantillon de validation.

Ensuite le plus gros fichier est utilisé pour évaluer l'impact de la taille de la base d'apprentissage.

Néanmoins, comme les résultats obtenus ne concurrencent pas ceux de MLlib et sont même assez *décevants* ([Besse et al. 2016](https://hal.archives-ouvertes.fr/hal-01350099)), ce calepin ne fait que décrire *a minima* le code de mise en oeuvre de la librairie sur les données MovieLens. 

In [ ]:
library(softImpute)

## 2  Etude de la matrice 100k
Les fichiers de données sont disponibles dans le répertoire [`data`](http://wikistat.fr/data) de [Wikistat](http://wikistat.fr/).

In [ ]:
# lecture
dBrut=read.csv("ml-ratings100k.csv")

In [ ]:
# extraction d'un échantillon test et d'un échantillon d'apprentissage
dTestInd=sample(nrow(dBrut),nrow(dBrut)/10,replace=FALSE)
dTest=dBrut[dTestInd,1:3]
dTrain=dBrut[-dTestInd,1:3]

In [ ]:
# Mise au format d'une matrice creuse
dTrainSparse=Incomplete(dTrain$userId,dTrain$movieId,dTrain$rating)
# appel de la fonctio
res=softImpute(dTrainSparse,rank.max=4,type="als",lambda=1,maxit=200)
# complétion
recom=impute(res,dTest[,1],dTest[,2])

In [ ]:
# Calcul de l'erreur (RMSE)
sqrt(mean((dTest[,3]-recom)**2))

## 3 Etude de la matrice complète

In [ ]:
# Lecture de la matrice
dBrut=read.csv("ratings20M.csv")

In [ ]:
# Extraction de l'échantillon test
dTestInd=sample(nrow(dBrut),nrow(dBrut)/10,replace=FALSE)
dTest=dBrut[dTestInd,1:3]
nrow(dTest)

In [ ]:
# sous-échantillonage de l'échantillon d'apprentissage
dInter=dBrut[-dTestInd,1:3]
taux=0.1
dTrainInd=sample(nrow(dInter),nrow(dInter)*taux,replace=FALSE)
dTrain=dInter[dTrainInd,1:3]
# Matrice d'échantillonnage sparse
dTrainSparse=Incomplete(dTrain$userId,dTrain$movieId,dTrain$rating)

**Attention**. La démarche adoptée ici pour trouver les meilleurs valeurs des paramètres (rang, pénalisation) est *abusive*. Les valeurs ont été optimisées sur l'échantillon test sans pour autant atteindre des résultats concurrentiels.

In [ ]:
# Factorisation
t1=Sys.time()
res=softImpute(dTrainSparse,rank.max=10,type="als",lambda=20,maxit=200)
t2=Sys.time()
# Reconstruction
recom=impute(res,dTest[,1],dTest[,2])
#RMSE
sqrt(mean((dTest[,3]-recom)**2))
difftime(t2,t1) 

Apprentissage avec fichier complet (taux=1): 20M de notes

rang | lambda | temps (') | rmse 
----|---------|-------|-----
 4  |  1       |  5.6 |  1.07   
 10 |  10  |  12.6 |  1.02  
 10 |  20  |  12.2 |  1.033 
 15 |  10  |  19.4 |  1.016
 20 |   1  |  26.9  | 1.02 
 20 |  10  |  26.1  | 1.016 
 20 |  15  |  24.4 |  1.018
 20 |  20  |  27.0  | 1.016 
 30 |  20  |  40.1 |  1.02


In [ ]:
# Tentative de post-traitement pour essayer d'améliorer les complétions.
recom[recom>5]=5
recom[recom<0]=0

In [ ]:
sqrt(mean((dTest[,3]-recom)**2))

In [ ]:
hist(recom)